# Proyecto individual: Sistema de recomendación de películas
​
Este proyecto constará de dos fases: `Ingenieria de datos`, `Modelamiento y evaluación con machine learning`.

### 1. Ingeniería de datos
* Esto incluye la limpieza y transformación de los datos, abordando problemas como:
    * valores faltantes, 
    * datos duplicados y variables irrelevantes,
    * valores anidados,
    * formateo de columnas,
    * nubes de palabras para ver las más frecuentes.
    * a fin de mejorar la calidad del dataset para el modelado.
    * análisis univariado.
    * análisis bivariado y multivariado.
​
### 2. Modelamiento y evaluación con machine learning
* Implementar un modelo de clasificación con aprendizaje supervisado que permita clasificar (**con un algoritmo de coseno de similitud, por ejemplo**) las películas por ... para encontrar una lista de 5 películas similares 

### 1. Análisis exploratorio de datos

#### 1.1. Importación de librerías

In [69]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Mostrar figuras de matplotlib en el entorno de Jupyter Notebook
%matplotlib inline

#### 1.2. Carga y visualización los datos.

In [70]:
df_movies = pd.read_csv('datasets/movies.csv')
df_movies.head(3)

C:\Users\Josue\AppData\Local\Temp\ipykernel_14172\2792449306.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('datasets/movies.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


#### 1.2. Eliminación de filas

In [71]:
df_movies_1 = df_movies[df_movies['original_language'] != 'en']

In [72]:
df_movies_1 = df_movies_1[df_movies_1['video'].notna()] 
df_movies_1 = df_movies_1[df_movies_1['status'].notna()] 


#### 1.3. Eliminación de columnas 

In [73]:
df_movies_2 = df_movies_1.drop(['belongs_to_collection', 'genres', 'poster_path', 'homepage', 'imdb_id', 'overview', 'production_companies', 'production_countries', 'original_language', 'runtime', 'spoken_languages', 'tagline', 'original_title', 'video'], axis=1)


In [74]:
df_movies_2.head()

,adult,budget,id,popularity,release_date,revenue,status,title,vote_average,vote_count
28,False,18000000,902,9.822423,1995-05-16,1738611.0,Released,The City of Lost Children,7.6,308.0
29,False,0,37557,1.100915,1995-04-30,0.0,Released,Shanghai Triad,6.5,17.0
32,False,0,78802,0.745542,1996-09-18,0.0,Released,Wings of Courage,6.8,4.0
52,False,0,49133,1.361286,1994-01-01,0.0,Released,Lamerica,7.7,11.0
57,False,0,11010,10.168437,1994-09-22,0.0,Released,The Postman,7.6,181.0


#### 1.4. Distinción entre day, month y year en date

In [75]:
# Convertir la columna 'date' a datetime
df_movies_2['release_date'] = pd.to_datetime(df_movies_2['release_date'], errors='coerce')

# Verificar el formato de la columna 'date' después de la conversión
print(df_movies_2['release_date'].head())

28   1995-05-16
29   1995-04-30
32   1996-09-18
52   1994-01-01
57   1994-09-22
Name: release_date, dtype: datetime64[ns]


In [76]:
# Crear nuevas columnas para día, mes y año
df_movies_2['day'] = df_movies_2['release_date'].dt.day
df_movies_2['month'] = df_movies_2['release_date'].dt.month
df_movies_2['year'] = df_movies_2['release_date'].dt.year

In [77]:
df_movies_2.head(2)

,adult,budget,id,popularity,release_date,revenue,status,title,vote_average,vote_count,day,month,year
28,False,18000000,902,9.822423,1995-05-16,1738611.0,Released,The City of Lost Children,7.6,308.0,16.0,5.0,1995.0
29,False,0,37557,1.100915,1995-04-30,0.0,Released,Shanghai Triad,6.5,17.0,30.0,4.0,1995.0


#### 1.5. Eliminación de la columna release_date y status

In [78]:
df_movies_2 = df_movies_2.drop(['release_date', 'status'], axis=1)

In [79]:
df_movies_2.head(2)

,adult,budget,id,popularity,revenue,title,vote_average,vote_count,day,month,year
28,False,18000000,902,9.822423,1738611.0,The City of Lost Children,7.6,308.0,16.0,5.0,1995.0
29,False,0,37557,1.100915,0.0,Shanghai Triad,6.5,17.0,30.0,4.0,1995.0


## 2.1. Creación de un endpoint de prueba

In [81]:
month_mapping = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

In [90]:
def film_month(df, month_name):
    # Convertir el nombre del mes a su número correspondiente
    month_number = month_mapping.get(month_name.lower())
    if month_number is None:
        raise ValueError(f"Mes '{month_name}' no es válido.")
    
    # Filtrar las filas que corresponden al mes especificado
    filtered_df = df_movies_2[df['month'] == month_number]
    # Contar los IDs en las filas filtradas
    count_ids = filtered_df['id'].count()
    return {"endpoint1": f"Fueron estrenadas {count_ids} peliculas en el mes de {month_name}"}

In [91]:
film_month(df=df_movies_2, month_name='enero')

{'endpoint1': 'Fueron estrenadas 1741 peliculas en el mes de enero'}